In [1]:
import pandas as pd

### Следующую часть можно пропустить и не нажимать, т.к. далее я использую csv файлы (я их пошлю), которые не содержат тегов. Не хотелось каждый раз всё заново пересчитывать.

In [2]:
# Файл train.csv содержит вакансии, которые имеют целевую переменную
train = pd.read_csv('train.csv', header=0, delimiter="\t", quoting=3)

#Файл test.csv содержит тексты, по которым нужно выдать предсказания
test = pd.read_csv('test.csv', header=0,  delimiter="\t", quoting=3)


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200067 entries, 0 to 200066
Data columns (total 4 columns):
id             200067 non-null object
name           200067 non-null object
description    200000 non-null object
target         199933 non-null float64
dtypes: float64(1), object(3)
memory usage: 6.1+ MB


In [6]:
#в трейне у каких-то вакансий не доставало таргета, поэтому я их убрала
traindataset = train.copy()
traindataset = traindataset[~traindataset['target'].isnull()]

In [7]:
#в тесте были вакансии только с id, но видимо без описаний и названий. Убираем их
testdataset = test.copy()
testdataset = testdataset[~testdataset['description'].isnull()]

In [8]:
testdataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170179 entries, 0 to 170235
Data columns (total 3 columns):
id             170179 non-null object
name           170179 non-null object
description    170179 non-null object
dtypes: object(3)
memory usage: 5.2+ MB


In [9]:
traindataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199933 entries, 0 to 200066
Data columns (total 4 columns):
id             199933 non-null object
name           199933 non-null object
description    199933 non-null object
target         199933 non-null float64
dtypes: float64(1), object(3)
memory usage: 7.6+ MB


In [10]:
from bs4 import BeautifulSoup
import re
def without_tags(s):
    return BeautifulSoup(s, 'lxml').text

In [11]:
# удаляем теги
# у меня это длится всё 7 минут примерно для каждого датасета.
traindataset['description']=traindataset['description'].map(without_tags)

In [12]:
%%time
testdataset['description']=testdataset['description'].map(without_tags)

Wall time: 6min 45s


In [21]:
filename = 'train_without_tags.csv'
traindataset.to_csv(filename,index=False, sep="\t")
print('Saved file: ' + filename)

Saved file: train_without_tags.csv


In [116]:
filename = 'test_without_tags.csv'
testdataset.to_csv(filename,index=False, sep="\t")
print('Saved file: ' + filename)

Saved file: test_without_tags.csv


#### Закончилась часть, которую можно пропустить! ) 

In [4]:
train_tags = pd.read_csv('train_without_tags.csv', header=0, delimiter="\t", quoting=3)

In [5]:
train_tags.head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на...,1.0
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0.0
2,2,Разработчик С++ (Криптограф),Требования: Опыт программирования на С++ Знан...,0.0
3,3,Фрезеровщик,Условия: На работу вахтовым методом в г. Калу...,0.0
4,4,Мерчендайзер/продавец-консультант,Компания Палладиум Стандарт - признанный лидер...,1.0


In [6]:
test_tags = pd.read_csv('test_without_tags.csv', header=0, delimiter="\t", quoting=3)

In [7]:
test_tags.head()

,id,name,description
0,200000,Дизайнер-консультант мебели,"Обязанности: Работа с клиентом в салоне,выезд..."
1,200001,Продавец-консультант (ТЦ на Пушкина),Обязанности: ∙ консультирование покупателей по...
2,200002,Менеджер по продажам,Торговый Дом «Форт» это ведущая компания Петер...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,"""Требуются продавцы консультанты в магазин жен..."
4,200004,Специалист по охране труда,Обязанности: осуществление контроля по соблю...


In [8]:
import re
#разд
def text_to_wordlist(text):   
    text = re.sub('[^a-zA-Zа-яA-Яё]', ' ', text)
    """на момент создания таргетов здесь я допустила ошибку и написала[^a-zA-Zа-яA-Zё]. 
    что ж.. я не учла все русские слова с верхним регистром.. ) Сейчас регулярка правильная, я так думаю.
    Но не учитывает цифры и слова с дефисом. Нужно ли это? не думаю. 
    К нижнему же регистру нужно было привести описания в момент удаления тегов.
    """
    words = text.lower().split()
    return words


In [9]:
# дальше мы формируем словарь для модели из теста и трейна
# я не стала строить предложения.. ) Элементами массива ArticleCorpus являются списки слов каждой статьи.
# я прочитала в stackoverflow, что так делают иногда! ) И это проще.. 
# Можно попробовать таки собрать список из предложений и посмотреть, что изменится!
# я думаю, что сюда также необходимо добавить 3ий csv файл!
%%time
ArticleCorpus = []
for review in train_tags["description"]:
    ArticleCorpus.append(text_to_wordlist(review))

Wall time: 51.7 s


In [10]:
%%time
for review in test_tags["description"]:
    ArticleCorpus.append(text_to_wordlist(review))

Wall time: 40 s


In [11]:
%%time
#тренеруем модель на все слова

# симортируем соответствующую функцию из модуля gensim, который должен быть установлен

from gensim.models.word2vec import Word2Vec 

# список параметров, которые можно менять по вашему желанию
num_features = 300  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(ArticleCorpus, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

D:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 17min 50s


In [12]:
model.init_sims(replace=True)

In [17]:
print(model['человек']) #А вот слова "Требуется" там нет, потому что верхний регистр = /

NameError: name 'model' is not defined

In [13]:
#здесь я просто еще раз формирую массив слов каждой статьи для теста и трейна. На этот раз отдельно.
#но надо было изначально формировать отдельно и только затем их сложить в один массив для обучения модели
#Я сделаю это позже! ) 
%%time
ArticleCorpusTrain = []

for review in train_tags["description"]:
    ArticleCorpusTrain.append(text_to_wordlist(review))

Wall time: 1min


In [14]:
%%time
ArticleCorpusTest = []

for review in test_tags["description"]:
    ArticleCorpusTest.append(text_to_wordlist(review))

Wall time: 56.1 s


In [15]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [16]:
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

In [17]:
#вектор трейна строим
train_vecs = texts_to_vecs(ArticleCorpusTrain, model, num_features)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [18]:
%%time
#вектор теста строим (1h 43min !) Для трейна время не меряла,но там еще больше.
test_vecs = texts_to_vecs(ArticleCorpusTest, model, num_features)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Wall time: 1h 43min 2s


In [24]:
len(train_tags["target"])

199933

In [25]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, train_tags["target"])


In [30]:
predict_proba = forest.predict_proba(test_vecs)

In [34]:
submission = pd.DataFrame({'id':test_tags['id'],'target':predict_proba[:,1]})
submission.head(10)

,id,target
0,200000,0.92
1,200001,1.00
2,200002,0.99
3,200003,0.97
4,200004,0.05
5,200005,0.00
6,200006,0.99
7,200007,0.28
8,200008,0.00
9,200009,0.02


In [35]:
test_tags.head(10)

,id,name,description
0,200000,Дизайнер-консультант мебели,"Обязанности: Работа с клиентом в салоне,выезд..."
1,200001,Продавец-консультант (ТЦ на Пушкина),Обязанности: ∙ консультирование покупателей по...
2,200002,Менеджер по продажам,Торговый Дом «Форт» это ведущая компания Петер...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,"""Требуются продавцы консультанты в магазин жен..."
4,200004,Специалист по охране труда,Обязанности: осуществление контроля по соблю...
5,200005,Эксперт по обеспечению качества при сооружении...,"Обязанности: Управление несоответствиями, в..."
6,200006,Торговый представитель (Арзамас),Обязанности: Ведение и развитие существующей ...
7,200007,Заместитель генерального директора по производ...,"""Обязанности: Доработка качества, в первую оч..."
8,200008,Backend Rust developer,Storiqa - это площадка для торговли физическим...
9,200009,Дизайнер-конструктор 3D,"""""""Обязанности: Адаптация дизайнов Визуализац..."


In [36]:
filename = 'targets.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: targets.csv
